# SCRAP BOOK - Please ignore


In [ ]:
df_pivot = df_day.pivot_table(index="day", columns="hour", values='minute', aggfunc='count')
display(df_pivot)

In [ ]:

       
# # -----------------------------------------------------------------------------
# def get_Tranformer(X):
#   numeric_features      = X.select_dtypes(include="number").columns
#   # categorical_features  = X.select_dtypes(exclude="number").columns

#   numerical_transformer = make_pipeline(
#       # SimpleImputer(strategy="median"),
#       StandardScaler(),
#   )

#   # categorical_transformer = make_pipeline(
#   #     SimpleImputer(strategy="most_frequent"),
#   #     OneHotEncoder(drop="first"),                 
#   # )

#   col_transformer = make_column_transformer(
#     (numerical_transformer,     numeric_features),
#     # (categorical_transformer, categorical_features),
#   )
#   return col_transformer


# wcss        = []
# nb_clstr    = []
# # silhouette  = []

# col_transformer = get_Tranformer(X)

# for n in range(2,20):
#   nb_clstr.append(n)
#   model = make_pipeline(
#     col_transformer,
#     KMeans(n_clusters= n, n_init='auto', random_state = k_random_state)        # default init = "k-means++", 
#   )
#   model.fit_transform(X)
#   # K-Means est le dernier élément du pipeline => utiliser -1
#   # .steps() renvoie des tuples (nom, objet)
#   # L'objet K-Means est accédé via [1]
#   kmeans_model = model.steps[-1][1] 
#   wcss.append(kmeans_model.inertia_)
#   print(f"WCSS for             K = {n:02} is wcss = {wcss[-1]:_.2f}")

#   # silhouette.append(silhouette_score(X, model.predict(X)))
#   # print(f"Silhouette score for K = {n:02} is {silhouette[-1]}")









In [ ]:
# df_april_04_2014 = df_april_04_2014.sort_values(by = 'hour')
fig = px.density_mapbox(
  df_day, 
  lat = "lat", 
  lon = "lon", 
  z = "hour", 
  radius = 10,  # default = 30
  # opacity = 0.05,
  mapbox_style="open-street-map", 
  color_continuous_scale=px.colors.sequential.Rainbow,
  zoom = 10.0,
  height = k_HeightPx,
  width = k_WidthPx,
  animation_frame = "hour",
  title= "april 04 2014 - Pickup per hour", 
)

fig.show()

In [ ]:
# df_wcss = pd.DataFrame(wcss)
# df_nb_clstr = pd.Series(nb_clstr)

# fig= px.line(
#     df_wcss,
#     x = df_nb_clstr,
#     y = df_wcss.iloc[:,-1]
# )

# fig.update_layout(
#     yaxis_title="Inertia",
#     xaxis_title="# Clusters",
#     title="Inertia per cluster"
# )
# fig.show() 

In [ ]:
# # Create a data frame
# cluster_scores=pd.DataFrame(silhouette)
# df_nb_clstr = pd.Series(nb_clstr)

# # Create figure
# fig = px.bar(data_frame=cluster_scores,
#              x=df_nb_clstr,
#              y=cluster_scores.iloc[:, -1]
#             )

# # Add title and axis labels
# fig.update_layout(
#     yaxis_title="Silhouette Score",
#     xaxis_title="# Clusters",
#     title="Silhouette Score per cluster"
# )

# # Render
# #fig.show(renderer="notebook")
# fig.show() # if using workspace

In [ ]:
fig = px.line(
    df_for_k_choice, 
    x=df_for_k_choice.index,
    y="wcss",
    labels={'wcss': 'WCSS'},  
)

fig.add_trace(
    go.Bar(
        x=df_for_k_choice.index,
        y=df_for_k_choice['sil'],
        name='Silhouette Score',
        yaxis='y2',  # Utilisation d'un deuxième axe Y
        opacity=0.6  # transparency 
    )
)

fig.update_layout(
    title_text=f"{date_string} - KMEANS - Determine optimal # of clusters (k)",
    xaxis_title="k",
    yaxis=dict(
        title="wcss",
    ),
    yaxis2=dict(
        title="Silhouette Score",
        overlaying='y',
        side='right'
    ),
    height=k_HeightPx,
    width=k_WidthPx,
)

fig.show()

In [ ]:
# Create a data frame
cluster_scores=pd.DataFrame(sil)
# k_frame = pd.Series(k)

# Create figure
fig = px.bar(
    cluster_scores,
    height = k_HeightPx,
    width = k_WidthPx,
    x=([i for i in range(2, 20)]),
    y=cluster_scores.iloc[:, -1],
    title="Silhouette Score per cluster"
)

# Add title and axis labels
fig.update_layout(
    yaxis_title="Silhouette Score",
    xaxis_title="# Clusters",
    
)

# Render
#fig.show(renderer="notebook")
fig.show() # if using workspace

In [ ]:
#HO ML V2 p 248
from sklearn.metrics import silhouette_samples
from matplotlib.ticker import FixedLocator, FixedFormatter

plt.figure(figsize=(11, 9))

for k in (3, 4, 5, 6):
    plt.subplot(2, 2, k - 2)
    
    y_pred = kmeans_per_k[k - 1].labels_
    silhouette_coefficients = silhouette_samples(X, y_pred)

    padding = len(X) // 30
    pos = padding
    ticks = []
    for i in range(k):
        coeffs = silhouette_coefficients[y_pred == i]
        coeffs.sort()

        color = mpl.cm.Spectral(i / k)
        plt.fill_betweenx(np.arange(pos, pos + len(coeffs)), 0, coeffs,
                          facecolor=color, edgecolor=color, alpha=0.7)
        ticks.append(pos + len(coeffs) // 2)
        pos += len(coeffs) + padding

    plt.gca().yaxis.set_major_locator(FixedLocator(ticks))
    plt.gca().yaxis.set_major_formatter(FixedFormatter(range(k)))
    if k in (3, 5):
        plt.ylabel("Cluster")
    
    if k in (5, 6):
        plt.gca().set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])
        plt.xlabel("Silhouette Coefficient")
    else:
        plt.tick_params(labelbottom=False)

    plt.axvline(x=silhouette_scores[k - 2], color="red", linestyle="--")
    plt.title("$k={}$".format(k), fontsize=16)

save_fig("silhouette_analysis_plot")
plt.show()


### Localization of the pick-ups during the day (per n-minute bin, no matter which day)
* For the value of n see the value of ``k_bin_len`` in the prelude cell of this notebook

In [ ]:

# ! One must order the dataframe by "bin" before the statement animation_frame='bin' otherwise the animation slider is really weird
df_month = df_month.sort_values(by= "bin", ascending=True)

fig = px.density_mapbox(
  df_month,
  lat="lat",
  lon="lon",
  animation_frame="bin", 
  mapbox_style="carto-positron",
  radius=5,                        # ! radius changed from 3 to 10
  zoom = 10.0,
  height = k_HeightPx,
  width = k_WidthPx,
  title= f"{date_string} - Localized pick-ups per bin of {k_bin_len} minutes over the selected month"
)

fig.show()

In [ ]:
# We focus on one month but since the process is heavy we only look at subset of data
# date_string = date_obj.strftime("%Y-%B")

# df_month  = df[(df["month"] == month) & (df["year"] == year)] 
# df_for_kmeans = df_month.sample(k_subset_size)
df_for_kmeans = df_all_same_weekday_same_hour.copy()
df_for_kmeans = df_for_kmeans[df_for_kmeans["bin"]==42]
df_for_kmeans.describe().T

In [ ]:
wcss = []
sil=[]
k=[]
for i in range(2, 11): # Z! 2 to 11
  kmeans = KMeans(n_clusters=i, random_state = k_random_state, n_init='auto') # default init='k-means++', n_init='auto' avoid warning
  kmeans.fit(df_for_kmeans)
  wcss.append(kmeans.inertia_)
  sil.append(silhouette_score(df_for_kmeans, kmeans.predict(df_for_kmeans)))
  k.append(i)

df_for_k_choice = pd.DataFrame({'k':k, 'wcss':wcss, 'sil':sil}).set_index('k')


In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df_for_k_choice.index,
        y=df_for_k_choice['wcss'],
        mode='lines+markers',
        name='WCSS',             # Ajout du nom dans la légende
        line=dict(color='blue')  # couleur pour la ligne
    )
)

fig.add_trace(
    go.Bar(
        x=df_for_k_choice.index,
        y=df_for_k_choice['sil'],
        name='Silhouette Score',
        yaxis='y2',
        opacity=0.6
    )
)

fig.update_layout(
    title_text=f"{date_string} - KMEANS - Determine optimal # of clusters (k)",
    xaxis_title="k",
    yaxis=dict(
        title="WCSS",
    ),
    yaxis2=dict(
        title="Silhouette Score",
        overlaying='y',
        side='right'
    ),
    height=k_HeightPx,
    width=k_WidthPx,
)

fig.show()

* On choisit un k qui se trouve au niveau du coude dans la courbe WCSS et qui correspond aussi à une valeur élevée du coefficient de silhouette. 
* On décide que k=5 

In [ ]:
df_with_kmeans_clusters = df_all_same_weekday_same_hour.copy()
df_with_kmeans_clusters = df_with_kmeans_clusters[df_with_kmeans_clusters["bin"]==42]
k_optimal = 5
kmeans = KMeans(n_clusters=k_optimal, random_state = k_random_state, n_init='auto') # default init='k-means++', n_init='auto' avoid warning 
df_with_kmeans_clusters['cluster'] = kmeans.fit_predict(df_with_kmeans_clusters)

In [ ]:
df_with_kmeans_clusters = df_with_kmeans_clusters.sort_values(by= "bin", ascending=True)

fig = px.scatter_mapbox(
  df_with_kmeans_clusters, 
  lat="lat", 
  lon="lon", 
  opacity = 0.5,
#   animation_frame='bin', 
  mapbox_style = "carto-positron",  
  color='cluster',
  zoom = 10.0,
  height = k_HeightPx,
  width = k_WidthPx,
  title = f"Localized pickups {date_obj.strftime('%A')} at {date_obj.strftime('%H')}H within the next {k_bin_len} minutes"
)

fig.show()

In [ ]:
fig = px.scatter_mapbox(
  df_hour_extended[df_hour_extended["cluster"]!=-1], 
  lat="lat", 
  lon="lon", 
  opacity = 0.5,
  # animation_frame='bin', 
  mapbox_style = "carto-positron",  
  color='cluster',
  zoom = 10.0,
  height = k_HeightPx,
  width = k_WidthPx,
  title = f"{date_string} - K-Means - Hot spots for the next hour",
)

fig.show()